# Action class

In [1]:
class Action:
    pre = {} #preconditions
    eff={} #effects
    cost=0
    name = ""
    def __init__(self, p, e, c, n):
        self.pre = p
        self.eff = e
        self.name = n
        self.cost = c

    def formatFluents(self, d):
      string = ""
      for k in d.keys():
        if(d[k]==0):
            string += "\n- " + k
        else:
            string += "\n+ " + k
      return string

    def __str__(self):
      return f'Name: {self.name} \nCost: {self.cost} \nPreconditions: {self.formatFluents(self.pre)} \nEffects: {self.formatFluents(self.eff)}'

## Define actions

In [2]:
a1 = Action(p={"person":1, "hungry":0, "hasCake":1},e={"hasCake":0, "person":0},c=2,n="EatCake B")
a2 = Action(p={"person":1, "hungry":1, "hasCake":1},e={"hasCake":0, "eatenCake":1,"hungry":0},c=2,n="EatCake A")
a3 = Action(p={"person":1,"hasMix":1},e={"hasCake":1,"hasMix":0},c=2,n="MakeCake")
a4 = Action(p={"person":1,"hungry":0},e={"hungry":1},c=4,n="Wait")
a5 = Action(p={"person":1},e={"hasMix":1},c=1,n="Go Shopping A")
a6 = Action(p={"person":1},e={"hasCar":1},c=1,n="Go Shopping B")

actions = [a1,a2,a3,a4,a5,a6]

In [3]:
goal = {'person':1,'hungry':0, "hasCake":1, "eatenCake":1}
start = {'person':1}

## Define functions

In [16]:
# Get only positive fluents
def getPositiveFluents(state):
    return {key:val for key, val in state.items() if val == 1}

# Check if two states match
def statesMatch(s1,s2):
    return s1 == s2

# Checks if an action is applicable for a given state
def isApplicable(state, action):
    positivesState = getPositiveFluents(state).copy()
    positivesAction = getPositiveFluents(action.pre).copy()
    return statesMatch(positivesState, positivesAction)
    

# For a state and all possible actions, return the [applicable actions] for the given state
def applicableActions(state, actions):
    arr = []
    for a in actions:
        if isApplicable(state, a):
            arr.append(a)
    return arr

# Given a state and an action, applies the action and returns the following state if all preconditions are met
def applyAction(state, action):
    if isApplicable(state, action) == False:
        raise Exception(f'Action {action.name} cannot be applied to state {str(state)}')
    for k in action.eff.keys():
        if k in state.keys():
            if (state[k]==1 and action.eff[k]==0):
                state.pop(k)
        else:
            state[k]=action.eff[k]
    return state

In [26]:
start = {'person':1}
applyAction(start, a6)

{'person': 1, 'hasCar': 1}

In [9]:
print([a.name for a in applicableActions(start, actions)])

['Wait', 'Go Shopping A', 'Go Shopping B']


# Testing

In [ ]:
def testStatesMatch():
    
    # Test null state
    s1 = {}
    s2 = {}
    assert(statesMatch(s1,s2))
    
    # Test equal state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':1,'hungry':0}
    assert(statesMatch(s1,s2))
    
    # Test non-equal fluents state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':0,'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':0}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':0}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    print(f'All test cases for function "statesMatch" passed')

    
def testGetPositiveFluents():
    
    s = {'person':0,'hungry':0}
    assert(getPositiveFluents(s) == {})
    
    s = {'person':1,'hungry':0}
    assert(getPositiveFluents(s) == {'person':1})
    
    s = {'person':1, 'hungry':1, 'angry': 1}
    assert(getPositiveFluents(s) == {'person':1, 'hungry':1, 'angry': 1})
    
    s = {'person':1}
    assert(getPositiveFluents(s) == {'person':1})
    
    print(f'All test cases for function "getPositiveFluents" passed')

In [ ]:
testStatesMatch()
testGetPositiveFluents()